In [ ]:
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

DB_PATH = "../data/bike_data.db"


In [ ]:
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql_query("""
    SELECT *
    FROM station_activity
    ORDER BY timestamp
""", conn)

conn.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])

df.head()


In [ ]:
df.shape


In [ ]:
station_id = df['station_id'].unique()[0]  # pick the first one

df_station = df[df['station_id'] == station_id]

plt.figure(figsize=(12,5))
plt.plot(df_station['timestamp'], df_station['free_bikes'])
plt.title(f"Bike Availability Over Time — Station {station_id}")
plt.xlabel("Time")
plt.ylabel("Free Bikes")
plt.grid(True)
plt.show()


In [ ]:
df_sorted = df.sort_values(["station_id", "timestamp"])
df_sorted['movement'] = df_sorted.groupby('station_id')['free_bikes'].diff().abs()

movement_df = (
    df_sorted.groupby("station_id")["movement"]
    .sum()
    .reset_index()
    .sort_values("movement", ascending=False)
)

movement_df.head(10)


In [ ]:
top10 = movement_df.head(10)

plt.figure(figsize=(12,6))
sns.barplot(data=top10, x='movement', y='station_id')
plt.title("Top 10 Most Active Stations (Bike Turnover)")
plt.xlabel("Total Movement")
plt.ylabel("Station ID")
plt.show()
